In [27]:
import sys
sys.path.insert(0, '../')

In [45]:
import json
import pandas as pd
import numpy as np
from src.agent import Pipeline, PipelineAgentConfig, IOAgent, AgentContext, registry
from src.llm_client import LLMClient
from src.schema.llm_config import LLMConfig
from src.utils.data import SyntheticEmotionDataset

In [29]:
with open("../configs/gpt_4o_mini_config.json", "r") as f:
    config_dct = json.load(f)
llm_config = LLMConfig.from_dict(config_dct)
llm_client = LLMClient(llm_config)

In [30]:
dset = SyntheticEmotionDataset(["../data/synthetic_dialogues/empathetic_dialogues.json",
                                "../data/synthetic_dialogues/non_empathetic_dialogues.json"],
                               [1, 0], "../data/synthetic_dialogues/scenarios.json")

In [31]:
gt_emotion = [dset[i][0].emotion.value for i in range(200)]

In [39]:
with open("../data/baseline_erc_results.json", "r") as f:
    data = json.load(f)["predictions"]
baseline_predicitons = [i['predicted'].split(';')[0].lower() for i in data]
with open("../data/inside_out_v1_erc_results.json", "r") as f:
    data = json.load(f)["predictions"]
inside_out_predictions = [i['predicted'].split(';')[0].lower() for i in data]
with open("../data/self_consistency_erc_results.json", "r") as f:
    data = json.load(f)["predictions"]
self_consistency_predictions = [i['predicted'].split(';')[0].lower() for i in data]

In [40]:
def accuracy(gt, pred):
    mask = [u == v for u, v in zip(gt, pred)]
    return np.array(mask).mean()

In [49]:
results = {
    "inside-out": accuracy(gt_emotion, inside_out_predictions),
    "baseline": accuracy(gt_emotion, baseline_predicitons), 
    "self consistency": accuracy(gt_emotion, self_consistency_predictions)
}
results_df = pd.DataFrame([results])
results_df

,inside-out,baseline,self consistency
0,0.59,0.475,0.475
